# Lab 02-ArcGIS 1
## Goals:
* **Downloads** .LAS files from MN DNR
* **Converts** the .LAS file into both a DEM and a TIN
* **Saves** the new DEM and TIN to disk
* **Exports** PDFs of the DEM and TIN with correct visualization

### Download .las files from MN DNR

In [1]:
# importing useful packages
import requests #will use to web scrape
import json
import pprint
import zipfile
import arcpy #use to export, convert, or save ?

#Note-to-self delete unnessacry packages and BeautifulSoup isn't available in ArcPy

In [2]:
# directing it to MN DNR website
base_url ="https://resources.gisdata.mn.gov/pub/data/elevation/lidar/examples/lidar_sample/las/"
filename = "4342-12-05.las"
page = requests.get(base_url + filename )


In [19]:
#downloading .las file?
open("4342-12-05.las", 'wb').write(page.content)
print("extracting the content...")
#pretty sure this is not in my 'Lab02_DNR_lidar' gdb

extracting the content...


In [5]:
#What is my current working directory
%pwd

'D:\\Fall 2021\\ArcGIS1\\Labs\\Lab02\\Lab02_DNR_lidar'

### Currently have the downloaded file, now need to convert the .las file to DEM and TIN

In [17]:
#setting the working directory to where my i want my data to go (.gdb folder)
arcpy.env.workspace = "D:\Fall 2021\ArcGIS1\Labs\Lab02\Lab02_DNR_lidar\Lab02_DNR_lidar.gdb"

#naming the path location to my downloaded las file
las = r"D:\Fall 2021\ArcGIS1\Labs\Lab02\Lab02_DNR_lidar\4342-12-05.las"

#create a las dataset instead of a feature class-need a las Dataset to make a DEM
arcpy.management.CreateLasDataset(las, r"D:\Fall 2021\ArcGIS1\Labs\Lab02\Lab02_DNR_lidar\LasDataset.lasd", "NO_RECURSION", None, "PROJCS['datum_D_North_American_1983_HARN_UTM_Zone_15N',GEOGCS['GCS_datum_D_North_American_1983_HARN',DATUM['D_unknown',SPHEROID['GRS_1980',6378137.0,298.257222101]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Transverse_Mercator'],PARAMETER['false_easting',500000.0],PARAMETER['false_northing',0.0],PARAMETER['central_meridian',-93.0],PARAMETER['scale_factor',0.9996],PARAMETER['latitude_of_origin',0.0],UNIT['Meter',1.0]],VERTCS['NAVD88 - Geoid03 (Meters)',VDATUM['unknown'],PARAMETER['Vertical_Shift',0.0],PARAMETER['Direction',1.0],UNIT['Meter',1.0]]", "COMPUTE_STATS", "ABSOLUTE_PATHS", "NO_FILES")

0


<Result 'D:\\Fall 2021\\ArcGIS1\\Labs\\Lab02\\Lab02_DNR_lidar\\LasDataset.lasd'>

### converting .las to TIN

In [15]:
#had to 'thin' the points because there were too many (only retained 70%)
arcpy.ddd.LasDatasetToTin(las, r"D:\Fall 2021\ArcGIS1\Labs\Lab02\Lab02_DNR_lidar\las_to_Tin", "RANDOM", "PERCENT", 70, 10000000, 1, "CLIP")

<Result 'D:\\Fall 2021\\ArcGIS1\\Labs\\Lab02\\Lab02_DNR_lidar\\las_to_Tin'>

### converting .las to DEM

In [24]:
# earlier I made lasDataset, which I need if I am to make a DEM, because the input must be a .lasd file
LasDataSet = r"D:\Fall 2021\ArcGIS1\Labs\Lab02\Lab02_DNR_lidar\LasDataset.lasd"

arcpy.conversion.LasDatasetToRaster(LasDataSet, r"D:\Fall 2021\ArcGIS1\Labs\Lab02\Lab02_DNR_lidar\las_to_DEM.tif", "ELEVATION", "BINNING AVERAGE LINEAR", "FLOAT", "CELLSIZE", 10, 1)

<Result 'D:\\Fall 2021\\ArcGIS1\\Labs\\Lab02\\Lab02_DNR_lidar\\las_to_DEM.tif'>

### Save both TIN and DEM to disk

In [25]:
#exporting raster DEM to gdb
arcpy.conversion.RasterToGeodatabase(r"'D:\Fall 2021\ArcGIS1\Labs\Lab02\Lab02_DNR_lidar\las_to_DEM.tif'", r"D:\Fall 2021\ArcGIS1\Labs\Lab02\Lab02_DNR_lidar\Lab02_DNR_lidar.gdb", '')

<Result ''>

In [26]:
#making a copy of las_to_TIN (saving it it my folder's project)
arcpy.ddd.CopyTin("las_to_TIN", r"D:\Fall 2021\ArcGIS1\Labs\Lab02\Lab02_DNR_lidar\C1_las_to_TIN", "CURRENT")

<Result 'D:\\Fall 2021\\ArcGIS1\\Labs\\Lab02\\Lab02_DNR_lidar\\C1_las_to_TIN'>

### export PDFs of TIN and DEM

In [30]:
#code from https://pro.arcgis.com/en/pro-app/latest/arcpy/mapping/introduction-to-arcpy-mp.htm

#locates my current project
aprx = arcpy.mp.ArcGISProject(r"D:\Fall 2021\ArcGIS1\Labs\Lab02\Lab02_DNR_lidar\Lab02_DNR_lidar.aprx")
#indentify the layout I have made within the project
DEM_pdf = aprx.listLayouts("DEM")[0]
#export the layout
DEM_pdf.exportToPDF(r"D:\Fall 2021\ArcGIS1\Labs\Lab02\Lab02_DNR_lidar\DEM.pdf", resolution = 300)

'D:\\Fall 2021\\ArcGIS1\\Labs\\Lab02\\Lab02_DNR_lidar\\DEM.pdf'

#### issue with the TIN PDF (it never opens, or is a blank document).
--guessing the issue is due to extreme file size (the data is too big to export in the PDF format). Workaround, try making a TIN dataset that is smaller.
-- ended up exporting the contour elevation of the lidar data instead of teh TIN as a subsitute

In [36]:
#replicate the process for the TIN
aprx = arcpy.mp.ArcGISProject(r"D:\Fall 2021\ArcGIS1\Labs\Lab02\Lab02_DNR_lidar\Lab02_DNR_lidar.aprx")
TIN_pdf = aprx.listLayouts("TIN")[0]
#export the layout (decreased resolution becasue the TIN sucks a lot of memory)
TIN_pdf.exportToPDF(r"D:\Fall 2021\ArcGIS1\Labs\Lab02\Lab02_DNR_lidar\TIN_50.pdf", resolution = 50)

'D:\\Fall 2021\\ArcGIS1\\Labs\\Lab02\\Lab02_DNR_lidar\\TIN_50.pdf'